In [1]:
# Importing packages
from time import time
import math
import numpy as np
import numba as nb
import pychange

%load_ext line_profiler

In [2]:
# Creating synthentic data
size = 500
k = 30
min_len = 10
max_cp = 40
pen = 100
test_series = np.hstack([np.random.normal(0, 1, (size,)),
                         np.random.normal(6, 1, (size,)),
                         np.random.normal(0, 2, (size,)),
                         np.random.normal(-4, 1, (size,)),
                         np.random.normal(3, 1, (size,)),
                         ] * 4)

In [77]:
@nb.experimental.jitclass([
    ('k', nb.int64),
    ('n', nb.int64),
    ('y', nb.float64[:, :]),
    ('c', nb.float64)
])
class NonParametricCost:
    
    def __init__(self, k):
        self.k = k
    
    def fit(self, x):
        self.n = x.shape[0]
        partial_sums = np.zeros(shape=(self.n + 1, self.k), dtype=x.dtype)
        sorted_data = np.sort(x)

        for i in np.arange(self.k):

            z = -1 + (2 * i + 1.0) / self.k
            p = 1.0 / (1 + (2 * self.n - 1) ** (-z))
            t = sorted_data[int((self.n - 1) * p)]

            for j in np.arange(1, self.n + 1):

                partial_sums[j, i] = partial_sums[j - 1, i]
                if x[j - 1] < t:
                    partial_sums[j, i] += 2
                if x[j - 1] == t:
                    partial_sums[j, i] += 1
        self.c = 2.0 * (-np.log(2 * self.n - 1)) / self.k
        self.y = partial_sums
        return self
    
    def cost(self, start, end):
        cost = 0.0
        ys = self.y[start, :]
        ye = self.y[end, :]
        for j in range(self.k):
            a_sum = ye[j] - ys[j]
            if a_sum != 0.0:
                diff = end - start
                a_half = 0.5 * a_sum
                if a_half != diff:
                    f = a_half / diff
                    fi = 1.0 - f
                    flog = math.log(f)
                    filog = math.log(fi)
                    t = f * flog
                    ti = fi * filog
                    l = t + ti
                    ld = diff * l
                    cost += ld
        return self.c * cost

In [24]:
@nb.njit(fastmath=True, nogil=True)
def sdt_segmentation(cost, min_size, jump, penalty, max_exp_cp):
    n_samples = cost.n
    times = np.arange(0, n_samples + jump, jump)
    times[-1] = n_samples

    min_idx_diff = math.ceil(min_size/jump)

    if len(times) <= min_idx_diff:
        return np.empty(0, dtype=np.int64)

    costs = np.full(len(times), np.inf)
    costs[0] = 0

    le = len(times) - min_idx_diff
    partitions = np.empty(le * max_exp_cp, dtype=np.int64)
    partition_starts = np.zeros(len(times) + 1, dtype=np.int64)

    start_idx = np.zeros(1, dtype=np.int64)
    for new_start, end_idx in enumerate(range(min_idx_diff, len(times))):
        new_costs = np.empty_like(start_idx, dtype=np.float64)
        for j, s in enumerate(start_idx):
            new_costs[j] = cost.cost(times[s], times[end_idx]) + penalty
        new_costs += costs[start_idx]
        best_idx = np.argmin(new_costs)
        best_cost = new_costs[best_idx]
        best_real_idx = start_idx[best_idx]
        if best_real_idx == 0:
            best_part = np.empty(0, dtype=np.int64)
        else:
            best_part_start = partition_starts[best_real_idx]
            best_part_end = partition_starts[best_real_idx+1]
            best_part = partitions[best_part_start:best_part_end]

        if end_idx == len(times) - 1:
            return times[best_part]

        new_partition = np.empty(len(best_part)+1, dtype=np.int64)
        new_partition[:-1] = best_part
        new_partition[-1] = end_idx

        new_part_start = partition_starts[end_idx]
        new_part_end = new_part_start + len(new_partition)

        while new_part_end > partitions.size:
            old_part = partitions
            partitions = np.empty(2 * old_part.size, dtype=np.int64)
            partitions[:old_part.size] = old_part

        partitions[new_part_start:new_part_end] = new_partition
        partition_starts[end_idx+1] = new_part_end

        costs[end_idx] = best_cost

        s2 = start_idx[new_costs <= best_cost + penalty]
        start_idx = np.empty(len(s2) + 1, dtype=np.int64)
        start_idx[:-1] = s2
        start_idx[-1] = new_start + 1

In [33]:
def binary_segmentation(cost, min_len, max_cps, penalty):
    """Runs binary segmentation on time series"""

    # Setting up summary statistics and objects
    n = cost.n
    is_candidate = np.arange(min_len, n - min_len)
    cps = np.zeros(shape=(n,))
    costs = np.full(shape=n, fill_value=0.0)
    cps[-1] = 1
    cps[0] = 1
    costs[-1] = cost.cost(0, n)

    # Iterating through changepoints until convergence
    while True:

        # Single Loop Iteration
        _cps = np.flatnonzero(cps)
        best_cand, best_cost, best_next_cost, best_next = 0, 0, 0, 0
        best_total_cost = costs.sum()

        # Looping over candidates
        for c1, c2 in np.stack((_cps[:-1], _cps[1:]), axis=-1):
            _cands = is_candidate[(is_candidate > c1) & (is_candidate < c2)]
            if _cands.shape[0] == 0:
                continue
            _costs = np.empty(shape=(_cands.shape[0], 3), dtype=np.float64)
            _other_costs = costs[: (c1 + 1)].sum() + costs[(c2 + 1):].sum()
            _costs[:, 0] = [cost.cost(c1, i) for i in _cands]
            _costs[:, 1] = [cost.cost(i, c2) for i in _cands]
            _costs[:, 2] = _costs[:, 0] + _costs[:, 1] + _other_costs + penalty
            _best_cand = np.argmin(_costs[:, 2])
            if _costs[_best_cand, 2] < best_total_cost:
                best_cand = _cands[_best_cand]
                best_cost = _costs[_best_cand, 0]
                best_next_cost = _costs[_best_cand, 1]
                best_total_cost = _costs[_best_cand, 2]
                best_next = c2

        if best_cand == 0:
            break
        else:
            cps[best_cand] = True
            costs[best_cand] = best_cost
            costs[best_next] = best_next_cost
            is_candidate[(best_cand - min_len): (best_cand + min_len)] = False
            if np.flatnonzero(cps).shape[0] > max_cp + 2:
                break
        
    return np.flatnonzero(cps)[1:-1]

In [45]:
# Replacing with jitclass
# double sigsq=(x2-((x*x)/n))/n;
#   if(sigsq<=0){sigsq=0.00000000001;}
#   return(n*(log(2*M_PI)+log(sigsq)+1));

@nb.experimental.jitclass([
    ('n', nb.typeof(test_series.shape[0])),
    ('y1', nb.typeof(np.append(0, test_series.cumsum()))),
    ('y2', nb.typeof(np.append(0, (test_series ** 2).cumsum())))
])
class NormalMeanVarCost:
    
    def __init__(self):
        pass
    
    def fit(self, x):
        self.n = x.shape[0]
        self.y1 = np.append(0, x.cumsum())
        self.y2 = np.append(0, (x ** 2).cumsum())
        return self
    
    def cost(self, start, end):
        
        n = end - start
        d1 = self.y1[end] - self.y1[start]
        d2 = self.y2[end] - self.y2[start]
        a1 = (d1 ** 2) / n
        a2 = d2 - a1
        a3 = a2 / n
        if a3 <= 0.0:
            a3 = 1e-8
        a4 = 2.8378771 + math.log(a3)
        return n * a4

In [58]:
@nb.experimental.jitclass([
    ('n', nb.int64),
    ('y1', nb.float64[:]),
    ('y2', nb.float64[:]),
])
class NormalMeanCost:
    
    def __init__(self):
        pass
    
    def fit(self, x):
        self.n = x.shape[0]
        self.y1 = np.append(0, x.cumsum())
        self.y2 = np.append(0, (x ** 2).cumsum())
        return self
    
    def cost(self, start, end):
        
        n = end - start
        d1 = self.y1[end] - self.y1[start]
        d2 = self.y2[end] - self.y2[start]
        a1 = (d1 ** 2) / n
        return d2 - a1

In [47]:
@nb.experimental.jitclass([
    ('n', nb.typeof(test_series.shape[0])),
    ('y1', nb.typeof(np.append(0, test_series.cumsum()))),
    ('y2', nb.typeof(np.append(0, (test_series ** 2).cumsum()))),
    ('y3', nb.typeof(np.append(0, ((test_series - test_series.mean()) ** 2).cumsum())))
])
class NormalVarCost:
    
    def __init__(self):
        pass
    
    def fit(self, x):
        self.n = x.shape[0]
        self.y1 = np.append(0, x.cumsum())
        self.y2 = np.append(0, (x ** 2).cumsum())
        self.y3 = np.append(0, ((x - x.mean()) ** 2).cumsum())
        return self
    
    def cost(self, start, end):
        
        n = end - start
        d1 = self.y1[end] - self.y1[start]
        d2 = self.y2[end] - self.y2[start]
        d3 = self.y3[end] - self.y3[start]
        a1 = math.log(d3 / n)
        a2 = 2.8378771 + math.log(a1)
        return n * a2

In [67]:
@nb.experimental.jitclass([
    ('n', nb.int64),
    ('y1', nb.float64[:]),
])
class PoissonMeanVarCost:
    
    def __init__(self):
        pass
    
    def fit(self, x):
        self.n = x.shape[0]
        self.y1 = np.append(0, x.cumsum())
        return self
    
    def cost(self, start, end):

        d1 = self.y1[end] - self.y1[start]
        if d1 == 0.0:
            return 0.0
        n = end - start
        a1 = math.log(n) - math.log(d1)
        return 2.0 * d1 * a1

In [42]:
@nb.njit(fastmath=True, nogil=True)
def segmentation(cost, min_len, jump, pen, max_cps):
    
    # Initializing parameters for segmentation
    n = cost.n
    cands = np.arange(0, n + jump, jump)
    cands[-1] = n
    n_cands = len(cands)
    min_cand_len = math.ceil(min_len / jump)
    
    # Initializing cost matrix
    costs = np.full(n_cands, fill_value=np.inf, dtype=np.float64)
    costs[0] = 0.0
    
    # Initializing partitions for cp locations
    n_cps = n_cands * (max_cps - min_cand_len)
    cps = np.empty(n_cps, dtype=np.int64)
    cps_starts = np.zeros(n_cands + 1, dtype=np.int64)
    r = np.array([0], dtype=np.int64)
    
    # Starting loop for search
    tau_star_range = range(min_cand_len, n_cands)
    for tau_ind, tau_star in enumerate(tau_star_range):
        
        # Initializing for cost calc
        r_len = len(r)
        f = np.empty(r_len, dtype=np.float64)
        
        # Calculating each candidate cost
        for j in range(r_len):
            tau = r[j]
            f[j] = cost.cost(cands[tau], cands[tau_star]) + costs[tau] + pen

        # Finding best candidate
        best_tau = np.argmin(f)
        best_cost = f[best_tau]
        best_r = r[best_tau]
        
        # Checking for zero condition
        if best_r == 0:
            r_part = np.empty(0, dtype=np.int64)
        else:
            r_start = cps_starts[best_r]
            r_end = cps_starts[best_r + 1]
            r_part = cps[r_start: r_end]
        
        # Checking if we are done segmenting
        if tau_star == n_cands - 1:
            return cands[r_part]
        
        # Updating changepoint partitions
        _part_len = len(r_part) + 1
        _part = np.empty(_part_len, dtype=np.int64)
        _part[: -1] = r_part
        _part[-1] = tau_star
        _part_start = cps_starts[tau_star]
        _part_end = _part_start + _part_len
        
        # Checking if we need to expand the max changepoint limit
        # TODO try replacing this with a typed list in numba
        while _part_end > n_cps:
            _cps = cps
            cps = np.empty(n_cps * 2, dtype=np.int64)
            cps[: n_cps] = _cps
            n_cps *= 2
            
        cps[_part_start: _part_end] = _part
        cps_starts[tau_star + 1] = _part_end
        
        costs[tau_star] = best_cost
        
        _r = r[f <= best_cost + pen]
        r = np.empty(len(_r) + 1, dtype=np.int64)
        r[: -1] = _r
        r[-1] = tau_ind + 1

In [70]:
cost = NormalMeanVarCost().fit(test_series)

In [50]:
cost = NormalVarCost().fit(test_series)

In [59]:
cost = NormalMeanCost().fit(test_series)

In [72]:
cost = PoissonMeanVarCost().fit(test_series)

In [79]:
cost = NonParametricCost(k).fit(test_series)

In [80]:
segmentation(cost, 10, 1, 100, 30)

array([ 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500,
       6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500], dtype=int64)

In [36]:
sdt_segmentation(cost, 10, 1, pen, 30)

array([ 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500,
       6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500], dtype=int64)

In [39]:
binary_segmentation(cost, 10, 30, pen)

array([ 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500,
       6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500], dtype=int64)

In [37]:
%timeit segmentation(cost, 10, 10, pen, 30)

5.62 ms ± 72.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
%timeit sdt_segmentation(cost, 10, 10, pen, 30)

5.77 ms ± 52.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
py_seg = segmentation.py_func

In [35]:
%lprun -f py_seg py_seg(cost, 10, 10, pen, 30)